In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup
import os
from datetime import datetime
import time

### We have already discussed GET requests. This scraper uses POST requests as scraping the DPCC site data needs a lot of form submissions. I welcome contributions to improve the explanations in this notebook and even additional contributions that demonstrate webscraping newspapers data using GET requests.

In [3]:
# Templates for searching
adv1Template = "http://www.dpccairdata.com/dpccairdata/display/AallAdvanceSearchCconc.php?stName="
adv2Template = "http://www.dpccairdata.com/dpccairdata/display/AallAdvanceSearchMet.php?stName="
adv1List = ['NH3','BEN','CO','NO2','NO1','NOX','O3','pXy','SO2','Tol']
adv2List = ['AT1','BP','PM10','PM25','RH','SR','WS','WD']

In [4]:
# More templates
stNamesDict = {
    "Pooth Khurd, Bawana" : "UG9vdGhLaHVyZEJhd2FuYQ==",
    "Nehru Nagar" : "TmVocnVOYWdhcg==",
    "Jawaharlal Nehru Stadium" : "SkxOU3RhZGl1bQ==",
    "Dr. Karni Singh Shooting Range" : "S2FybmlTaW5naFNob290aW5nUmFuZ2U=",
    "Major Dhyan Chand National Stadium" : "TmF0aW9uYWxTdGFkaXVt",
    "Patparganj" : "UGF0cGFyZ2Fuag==",
    "Vivek Vihar" : "Vml2ZWtWaWhhcg==",
    "Sonia Vihar" : "U29uaWFWaWhhcg==",
    "Narela" : "TmFyZWxh",
    "Najafgarh" : "TmFqYWZnYXJo",
    "Rohini" : "Um9oaW5pU2VjdG9yMTY=",
    "Okhla Phase-2" : "T2tobGFQaGFzZTI=",
    "Ashok Vihar" : "QXNob2tWaWhhcg==",
    "Wazirpur" : "V2F6aXJwdXI=",
    "Jahangirpuri" : "SmFoYW5naXJwdXJp",
    "Dwarka, Sector 8" : "RHdhcmthU2VjdHJvOA==",
    "Pusa" : "UHVzYQ==",
    "Sri Auribindo Marg" : "U3JpQXVyYmluZG9NYXJn",
    "Mundka" : "TXVuZGth",
    "Anand Vihar" : "QW5hbmRWaWhhcg==",
    "Mandir Marg" : "TWFuZGlybWFyZw==",
    "Punjabi Bagh" : "UHVuamFiaUJhZ2g=",
    "R.K. Puram" : "UktQdXJhbQ=="
}

In [5]:
curr_dir = os.getcwd()

### This piece of code below badly needs documentation. I leave it up to you guys to test your open source contribution skills and document it.

### Running the following code will start scraping DPCC data and this code block takes a lot of time to execute. Give or take, it might go up till 2 or 3 days. Hit the stop button on toolbar if you run it and want to stop it, attempt to debug it by adding print statements in between etc... and figure out what is going on.

In [ ]:
for key, value in stNamesDict.items():

    print(key)
    save_dir = os.path.join(curr_dir,key)
    os.makedirs(save_dir, exist_ok=True)
    os.chdir(save_dir) #CHANGE dir
#     print(os.getcwd())
    uri = adv1Template+value
    headers = {
        'Origin': 'http://www.dpccairdata.com',
        'Upgrade-Insecure-Requests':'1',
        'Content-Type':'application/x-www-form-urlencoded',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36',
        'X-DevTools-Emulate-Network-Conditions-Client-Id':'2B8F1DB78308223222DE718C809C036C',
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Referer': uri,
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'en',
        'Cookie':'PHPSESSID=8433578c40d503e4f046f054f60c531b'
    }
    
    for param in adv1List:
        
        print(param)
        l=[]
        start = '2012-01-01 00:00' #CHANGE start date and time
        endSec = datetime.strptime(start,'%Y-%m-%d %H:%M').timestamp()
        while endSec<=time.time()-302400:
            print(start)
            endSec = datetime.strptime(start,'%Y-%m-%d %H:%M').timestamp() +604800-1 #end date and time set to 1 week ahead
            end = datetime.fromtimestamp(endSec)
            end = end.strftime('%Y-%m-%d %H:%M')
            formFields = (
                (r'parameters',param),
                (r'fDate',start),
                (r'eDate',end),
                (r'duration',r'1'),
                (r'graphType',r'Line'),
                (r'submit',r'Search')
            )

            formDict = {}
            for a,b in formFields:
                formDict[a] = b

            session = requests.Session()
            resp = session.post(uri,headers=headers,data=formDict)
            session.close()

            spG = soup(resp.content,'lxml')
            sps = spG.findAll('script')
            spF = sps[-1]
            spFF = str(spF)[str(spF).find('<graph caption'):str(spF).find('</graph>')+8]
            asd = soup(spFF,'lxml').findAll('set')
            for a in asd:
                q = soup(str(a),'lxml').find('set')['name']
                q = q.split(" ")
                w = soup(str(a),'lxml').find('set')['value']
                l.append([q[0],q[1],w])
            start = datetime.fromtimestamp(endSec+1) #start date and time set to 1 week ahead of previous
            start = start.strftime('%Y-%m-%d %H:%M')
#         print(l)
        df = pd.DataFrame(l,columns = ['Date','Timestamp','value'])
        df.to_csv(key+"_"+param+".csv") #store to CSV


In [51]:
# This is a function I had used to clean the data after scraping it
def cvrt(ts):
    try:
        return pd.to_datetime(ts)
    except:
        return pd.to_datetime(ts,format = '%m/%d/%Y %H:%M:%S')

In [ ]:
# Same: This is a function I had used to clean the data after scraping it
def cleanDF2(df):
    df['time'] = df['Date'] +" "+ df['Timestamp']
    df['time'] = pd.to_datetime(df['time'],format = '%m/%d/%Y %H:%M:%S')
    df = df.set_index('time')
    df = df['value']
    return df

In [22]:
def is_number(n):
    try:
        float(n)   # Type-casting the string to `float`.
                   # If string is not a valid `float`, 
                   # it'll raise `ValueError` exception
    except:
        return False
    return True

In [10]:
def getDir(key):
    return os.path.join(curr_dir,key)

In [ ]:
# Run this code at your own risk. I haven't debugged it. Again, I leave it upto you guys to download the data,
# figure out why the data needs to be cleaned and run this code block to see what it does
for key in stNamesDict:
    df = pd.DataFrame(columns=cols)
    for item in cols:
        print(item)
        dftemp = cleanDF(pd.read_csv(getDir(key,item)))
        for i,t in dftemp.iteritems():
            if is_number(t):
                t = float(t)
            else:
                t = np.nan
            i = cvrt(i)
            df.loc[i,item] = t
        df[item] = pd.to_numeric(df[item])
        assert(df[item].dtype != 'object')
    df = df.sort_index()
    df.to_csv('Final/'+key+'.csv')
    doneSt.append(key)
    print(key)

## I hope you guys enjoyed the workshops! Contact me if you want any other aspect you want covered in another class that you feel that I could live upto your expectations for and I will do my best! But for now, Good Luck!